In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as img
import numpy as np
import pandas as pd
import torch
import random
import wandb
import torch.nn as nn

In [2]:
if torch.cuda.is_available():
    device_name = torch.device("cuda")
else:
    device_name = torch.device('cpu')
print("Using {}.".format(device_name))

Using cuda.


In [3]:
def preprocessingData(df):
    eng_maxlen = len(max(df['eng'], key=len))
    hin_maxlen = len(max(df['hin'], key=len))
    max_len = max(eng_maxlen, hin_maxlen)
    eng_words = df['eng'].copy()
    for i in range(len(eng_words)):
      l = len(eng_words[i])
      eng_words[i] = eng_words[i] + "*"*(max_len - l + 3)
    hin_words = df['hin'].copy()
    for i in range(len(hin_words)):
      l = len(hin_words[i])
      hin_words[i] = "#" + hin_words[i] + "*"*(max_len - l + 2)
    unique_eng_letters = set(''.join(eng_words))
    unique_hin_letters = set(''.join(hin_words))
    int_to_eng = dict(enumerate(unique_eng_letters))
    eng_to_int = {char: ind for ind, char in int_to_eng.items()}

    int_to_hin = dict(enumerate(unique_hin_letters))
    hin_to_int = {char: ind for ind, char in int_to_hin.items()}
    
    index_eng_words = []
    for eng_word in eng_words:
      index_eng_word = [eng_to_int[i] for i in eng_word]
      index_eng_words.append(index_eng_word)
    index_hin_words = []
    for hin_word in hin_words:
      index_hin_word = [hin_to_int[i] for i in hin_word]
      index_hin_words.append(index_hin_word)
    tensor_eng = torch.tensor(index_eng_words).to(device_name)
    tensor_hin = torch.tensor(index_hin_words).to(device_name)
    return tensor_eng, tensor_hin, int_to_eng, int_to_hin, unique_eng_letters, unique_hin_letters


In [4]:
class GRU_Encoder(nn.Module):
  def __init__(self, input_size, hid_size, num_of_enc_layers, emb_size, batch_size, dropout, bi_direct):
    super(GRU_Encoder, self).__init__()
    self.input_size = input_size
    self.hid_size = hid_size
    self.num_of_enc_layers = num_of_enc_layers
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.bi_direct = bi_direct
    self.dropout = dropout
    self.embedding = nn.Embedding(input_size, emb_size)
    # print("IS:{} ES:{}".format(input_size, emb_size))
    self.gru = nn.GRU(emb_size, hid_size, num_of_enc_layers, bidirectional = bi_direct, dropout = dropout)

  def forward(self, input_data, hidden):
    input_data = input_data.T
    # print(input_data.shape)
    embed = self.embedding(input_data).to(device_name)
    # print(embed.shape,hidden.shape)
    # embed = embed.view(-1, self.batch_size, self.hid_size)
    output, hidden = self.gru(embed, hidden)
    # if(self.bi_direct):
    #   print("bir\n")
    #   hidden = hidden.resize(2, self.num_of_enc_layers, self.batch_size, self.hid_size)
    #   print(hidden.shape)
    #   hidden = torch.add(hidden[0], hidden[1])/2
    #   print(hidden.shape)
    return output, hidden
  
  def initialiseHidden(self):
    if(self.bi_direct):
      return torch.zeros(2*self.num_of_enc_layers, self.batch_size, self.hid_size, device = device_name)
    else:
      return torch.zeros(self.num_of_enc_layers, self.batch_size, self.hid_size, device = device_name)
  

In [5]:
class GRU_Decoder(nn.Module):
  def __init__(self, op_size, num_of_dec_layers, hid_size, batch_size, emb_size, dropout, bi_direct):
    super(GRU_Decoder, self).__init__()
    self.op_size = op_size
    self.hid_size = hid_size
    self.num_of_dec_layers = num_of_dec_layers
    self.emb_size = emb_size
    self.batch_size = batch_size
    self.bi_direct = bi_direct
    self.embedding = nn.Embedding(op_size, emb_size)
    self.op = nn.Linear(2*hid_size, op_size) if (bi_direct) else nn.Linear(hid_size, op_size)
    self.softmax = nn.LogSoftmax(dim = 2)
    self.gru = nn.GRU(emb_size, hid_size, num_of_dec_layers, bidirectional = bi_direct, dropout = dropout)

  def forward(self, input_data, hidden):
    # print(input_data)
    embed = self.embedding(input_data)
    embed = embed.view(-1, self.batch_size, self.emb_size)
#     print(hidden.shape)
    out, hidden = self.gru(embed, hidden)
    # print(out.shape)
    temp = self.op(out)
    out = self.softmax(temp)
    return out, hidden

In [104]:
def train(input_data, target_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, bi_direct, cell_type):
  teacher_forcing = 0.5
  loss = 0
  total = 0
  c = 0
  
  for b in range(0, len(input_data), batch_size):
    
    x, y = input_data[b : b+batch_size], target_data[b : b+batch_size]
    temp = 0
    enc_optimizer.zero_grad()
    dec_optimizer.zero_grad()
    # x = x.T
    # y = y.T
    if(cell_type == 'GRU'):
      enc_hidden = encoder.initialiseHidden()
      enc_output, enc_hidden = encoder(x, enc_hidden)
#       print("AFT_Encoder Hidden : {}".format(enc_hidden.shape))
#       if(num_of_dec_layers > num_of_enc_layers):
# #         print("1")
#         num = num_of_dec_layers-2
#         dec_hidden = enc_hidden
#         while(num != num_of_enc_layers):
#           dec_hidden = torch.cat([dec_hidden, enc_hidden[-1].unsqueeze(0)], dim = 0)
#           num -= 1
#       elif(num_of_dec_layers < num_of_enc_layers):
# #         print("2")
#         dec_hidden = enc_hidden[-num_of_dec_layers:]
#       else:
# #         print("3")
#         dec_hidden = enc_hidden
      dec_hidden = enc_hidden[-1].repeat(num_of_dec_layers, 1, 1)
      if bi_direct:
        dec_hidden = dec_hidden.repeat(2,1,1)
      y = y.T
      dec_input = y[0]
#       print("AFT_Decoder Hidden : {}".format(dec_hidden.shape))
      condition = False if random.random() > teacher_forcing else True
      if(condition):
        for i in range(len(y)):
          dec_output, dec_hidden = decoder(dec_input, dec_hidden)
          temp = loss_fn(torch.squeeze(dec_output), y[i])
          loss += temp.item()
          c += 1
          dec_input = y[i]
      else:
        for i in range(len(y)):
          dec_output, dec_hidden = decoder(dec_input, dec_hidden)
          prob, idx = dec_output.topk(1)
          temp = loss_fn(torch.squeeze(dec_output), y[i])
          loss += temp.item()
          c += 1
#           print(idx)
          dec_input = idx
    temp.backward()
    enc_optimizer.step()
    dec_optimizer.step()
    print("\tRunning loss",loss/c)
#     total += 1
#     loss += (temp/c)
    
  # print(len(target_data))
  return loss/c, encoder, decoder



In [105]:
def eval(input_data, target_data, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, cell_type):
  out = []
  for b in range(0, len(input_data), batch_size):
    x, y = input_data[b : b+batch_size], target_data[b : b+batch_size]
    encoder.eval()
    decoder.eval()
    predicted_data = list()
    # x = x.T
    # y = y.T
    if(cell_type == 'GRU'):
      enc_hidden = encoder.initialiseHidden()
      enc_output, enc_hidden = encoder(x, enc_hidden)
    
      
#       if(num_of_dec_layers > num_of_enc_layers):
#         num = num_of_dec_layers
#         dec_hidden = enc_hidden
#         while(num != num_of_enc_layers):
#           dec_hidden = torch.cat([dec_hidden, enc_hidden[-1].unsqueeze(0)], dim = 0)
#           num -= 1
#       elif(num_of_dec_layers < num_of_enc_layers):
#         dec_hidden = enc_hidden[-num_of_dec_layers:]
#       else:
#         dec_hidden = enc_hidden
      y = y.T
      
      dec_hidden = enc_hidden[-1].repeat(num_of_dec_layers,1,1)
      dec_input = y[0]

      for i in range(len(y)):
        dec_output, dec_hidden = decoder(dec_input, dec_hidden)
        prob, idx = dec_output.topk(1)
        idx = idx.squeeze()
        dec_input = idx
        predicted_data.append(idx.tolist())
      out.append(predicted_data)
  return out


In [106]:
def training(input_data, input_size, target_data, target_size, max_input_size, epochs, batch_size, emb_size, num_of_enc_layers, num_of_dec_layers, hid_size, cell_type, bi_direct, enc_dropout, dec_dropout, beam_size):
  learning_rate = 0.001
  if(cell_type == "GRU"):
    encoder = GRU_Encoder(input_size, hid_size, num_of_enc_layers, emb_size, batch_size, enc_dropout, bi_direct).to(device_name)
    decoder = GRU_Decoder(target_size, num_of_dec_layers, hid_size, batch_size, emb_size, dec_dropout, bi_direct).to(device_name)
  
  enc_optimizer = torch.optim.Adam(encoder.parameters(), learning_rate)
  dec_optimizer = torch.optim.Adam(decoder.parameters(), learning_rate)
  loss_fn = nn.CrossEntropyLoss(reduction = 'sum')
  encoder.train()
  decoder.train()
  loss_list = []
  for i in range(epochs):
    loss, encoder, decoder = train(input_data, target_data, loss_fn, enc_optimizer, dec_optimizer, encoder, decoder, num_of_enc_layers, num_of_dec_layers, batch_size, bi_direct, cell_type)
    loss_list.append(loss/51200)
    print("Epoch : {} \tLoss : {}".format(i, loss))

  return encoder, decoder, num_of_enc_layers, num_of_dec_layers


In [107]:
def calculateAccuracy(y_pred, y_true):
  cnt = 0
  for i,j in zip(y_pred, y_true):
    cor = torch.eq(i, j)
    if(torch.mean(cor.float()).item() == 1.0):
      cnt += 1
  return cnt / len(y_pred)

In [108]:
df = pd.read_csv('/kaggle/input/fdl-a3/hin_train.csv', names=['eng','hin'])
tensor_eng, tensor_hin, int_to_eng, int_to_hin, unique_eng_letters, unique_hin_letters = preprocessingData(df)

In [109]:
input_data = tensor_eng
input_size = len(unique_eng_letters)
target_data = tensor_hin
target_size = len(unique_hin_letters) 
max_input_size = tensor_eng.shape[1] 
epochs = 7
batch_size = 64 
emb_size = 256 
num_of_enc_layers = 2
num_of_dec_layers = 3
hid_size = 256
cell_type = "GRU" 
bi_direct = True 
enc_dropout = 0.3
dec_dropout = 0.3 
beam_size = 1

In [ ]:
encoder, decoder, num_of_enc_layers, num_of_dec_layers = training(input_data, input_size, target_data, target_size, max_input_size, epochs, batch_size, emb_size, num_of_enc_layers, num_of_dec_layers, hid_size, cell_type, bi_direct, enc_dropout, dec_dropout, beam_size)

In [ ]:
trained_pred = eval(tensor_eng, tensor_hin, encoder, decoder, num_of_enc_layers, num_of_dec_layers, 64, "GRU")

In [23]:
out = []
ten_pred = torch.tensor(trained_pred)
for i in range(len(trained_pred)):
    temp = ten_pred[i].T
    out.extend(temp)
out_pred = torch.stack(out).to(device_name)
print(calculateAccuracy(out_pred, tensor_hin))

0.41748046875


In [54]:
df_test = pd.read_csv('/kaggle/input/fdl-a3/hin_test.csv', names=['eng','hin'])

eng_words = df_test['eng'].copy()
for i in range(len(eng_words)):
    l = len(eng_words[i])
    eng_words[i] = eng_words[i] + "*"*(24 - l + 3)
hin_words = df_test['hin'].copy()
for i in range(len(hin_words)):
    l = len(hin_words[i])
    hin_words[i] = "#" + hin_words[i] + "*"*(24 - l + 2)
    
eng_to_int = {v: k for k, v in int_to_eng.items()}
hin_to_int = {v: k for k, v in int_to_hin.items()}
hin_to_int['_'] = len(hin_to_int)

In [56]:
index_eng_words = []
for eng_word in eng_words:
    index_eng_word = [eng_to_int[i] for i in eng_word]
    index_eng_words.append(index_eng_word)
index_hin_words = []
for hin_word in hin_words:
    index_hin_word = [hin_to_int[i] if i in hin_to_int else hin_to_int['_'] for i in hin_word]
    index_hin_words.append(index_hin_word)

tensor_eng_test = torch.tensor(index_eng_words).to(device_name)
tensor_hin_test = torch.tensor(index_hin_words).to(device_name)

trained_pred = eval(tensor_eng_test, tensor_hin_test, encoder, decoder, num_of_enc_layers, num_of_dec_layers, 64, "GRU")


In [58]:
torch.tensor(trained_pred)

tensor([[[34, 34, 34,  ..., 34, 34, 34],
         [51, 32,  3,  ...,  3, 25, 63],
         [37, 11, 10,  ..., 11, 58, 65],
         ...,
         [ 9,  9,  9,  ...,  9,  9,  9],
         [ 9,  9,  9,  ...,  9,  9,  9],
         [ 9,  9,  9,  ...,  9,  9,  9]],

        [[34, 34, 34,  ..., 34, 34, 34],
         [24, 65, 24,  ..., 43, 41, 56],
         [38, 45,  1,  ..., 52, 38, 11],
         ...,
         [ 9,  9,  9,  ...,  9,  9,  9],
         [ 9,  9,  9,  ...,  9,  9,  9],
         [ 9,  9,  9,  ...,  9,  9,  9]],

        [[34, 34, 34,  ..., 34, 34, 34],
         [ 3, 24, 39,  ..., 43, 21, 63],
         [14, 65, 38,  ..., 52, 18,  1],
         ...,
         [ 9,  9,  9,  ...,  9,  9,  9],
         [ 9,  9,  9,  ...,  9,  9,  9],
         [ 9,  9,  9,  ...,  9,  9,  9]],

        ...,

        [[34, 34, 34,  ..., 34, 34, 34],
         [44, 37, 43,  ..., 63, 30, 32],
         [ 2, 57, 11,  ..., 38,  5, 38],
         ...,
         [ 9,  9,  9,  ...,  9,  9,  9],
         [ 9,  9,  9, 

In [65]:
out = []
ten_pred = torch.tensor(trained_pred)
for i in range(len(trained_pred)):
    temp = ten_pred[i].T
    out.extend(temp)
out_pred = torch.stack(out).to(device_name)
print(calculateAccuracy(out_pred, tensor_hin_test))

0.323974609375


In [ ]:
pred_up[0].shape

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/FDL_A3/hin/hin_test.csv', names=['eng','hin'])
eng_words = df_test['eng']
maxlen = len(max(df['eng'], key=len))
index_eng_words = []
for eng_word in eng_words:
  index_eng_word = [eng_to_int[i] for i in eng_word]
  l = len(index_eng_word)
  index_eng_word.extend([0]*(maxlen-l+2))
  index_eng_words.append(index_eng_word)

tensor_eng_test = torch.tensor(index_eng_words).to(device_name)
tensor_hin_test = torch.tensor(index_hin_words).to(device_name)

In [ ]:
pred_value = eval(tensor_eng_test, tensor_hin_test, )

# **Experimental Area**

In [44]:
l = [i if i!=5 else 99 for i in range(10)]
l

[0, 1, 2, 3, 4, 99, 6, 7, 8, 9]

In [63]:
ten_pred = torch.tensor(trained_pred)
ten_pred[0].T.shape

torch.Size([64, 27])

In [137]:
a = torch.tensor([[[1,2,3],[1,2,3]], [[1,2,3],[1,2,3]]])
print(a.shape)
a = a.repeat(2,1,1)
print(a.shape)

torch.Size([2, 2, 3])
torch.Size([4, 2, 3])


In [64]:
out_pred = ten_pred[0].T
num=0
while num <10:
    for i in out_pred[num]:
        print(int_to_hin[i.item()],end="")
    print("\n")
    num+=1

#थर्मैक्स******************

#सिखाएगा*******************

#लीयर्न********************

#ट्विटर्स******************

#तिरुनेलवली****************

#इंडेपेंडेंस***************

#स्पेषियों*****************

#शुरूह*********************

#कोल्हापुर*****************

#अझर***********************



In [ ]:
for i in df['eng']:
  if(len(i) > 24):
    print(i)

In [ ]:
a = [1,2,3,4]
a.append(0*4)
a

In [ ]:
pred_up

In [ ]:
trained_pred = torch(tensor(trained_pred))

In [ ]:
for i in temp[0]:
    print(int_to_hin[i.item()], end="")

In [ ]:
for i in temp:
  for j in i:
    print(int_to_hin[j.item()], end="")
  print("\n")

In [ ]:
for x, y in zip(tensor_eng, tensor_hin):
  print(x,y)

In [ ]:
len(unique_hin_letters)

In [ ]:
a = [1,2],[3,4]
b = [1,1],[3,4]
ta = torch.tensor(a)
tb = torch.tensor(b)
calculateAccuracyUPD(a,b)